In [194]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

In [195]:
train_data = pd.read_csv(f'D:/Download/Training set.csv')
train_data['Sex'] = train_data['Sex'].replace({'Female':1,'Male':0})

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15740\3608832630.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['Sex'] = train_data['Sex'].replace({'Female':1,'Male':0})


In [196]:
male_data = train_data[train_data['Sex'] == 0]
female_data = train_data[train_data['Sex'] == 1]

In [197]:
male_mean = [male_data.iloc[:,0].mean(),male_data.iloc[:,1].mean()]
male_cov = np.cov(male_data.iloc[:, 0], male_data.iloc[:, 1])

In [198]:
female_data = female_data[female_data['Height'] <= 250]
female_mean = [female_data.iloc[:,0].mean(),female_data.iloc[:,1].mean()]
female_cov = np.cov(female_data.iloc[:, 0], female_data.iloc[:, 1])
f_prior_p = len(female_data)/len(train_data)
m_prior_p = 1-f_prior_p

In [199]:
class GaussianModel:
    def __init__(self, male_mean, female_mean, male_cov, female_cov, prior_prob_m, prior_prob_f):
        self.male_m = np.array(male_mean)
        self.female_m = np.array(female_mean)
        self.male_cov = np.array(male_cov)
        self.female_cov = np.array(female_cov)
        self.prior_prob_m = prior_prob_m
        self.prior_prob_f = prior_prob_f

    def predict(self, x):
        x = np.array(x) 
        result = []
        male_cov_det = np.linalg.det(self.male_cov)
        female_cov_det = np.linalg.det(self.female_cov)
        male_cov_inv = np.linalg.inv(self.male_cov)
        female_cov_inv = np.linalg.inv(self.female_cov)
        for row in x:
            diff_m = row - self.male_m
            m_density = (
                1 / np.sqrt((2 * np.pi) ** len(row) * male_cov_det)
                * np.exp(-0.5 * diff_m @ male_cov_inv @ diff_m)
            )
            diff_f = row - self.female_m
            f_density = (
                1 / np.sqrt((2 * np.pi) ** len(row) * female_cov_det)
                * np.exp(-0.5 * diff_f @ female_cov_inv @ diff_f)
            )
            pro_m = self.prior_prob_m * m_density
            pro_f = self.prior_prob_f * f_density
            if pro_m < pro_f:
                result.append(1)  # Female
            else:
                result.append(0)  # Male
        return result

In [200]:
test_data = pd.read_csv(f'D:/Download/Test set.csv')
test = test_data.iloc[:,:2]
labels = test_data.iloc[:,2]
labels = labels.replace({'Female':1,'Male':0})
labels = np.array(labels)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15740\2384424853.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels = labels.replace({'Female':1,'Male':0})


array([1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1], dtype=int64)

In [205]:
model = GaussianModel(male_mean,female_mean,male_cov,female_cov,m_prior_p,f_prior_p)
prediction = model.predict(test)
from sklearn.metrics import confusion_matrix
confusion_matrix(labels,prediction)

array([[94, 12],
       [15, 84]], dtype=int64)